In [1]:
# Gives a high level interface to grocery list making.

In [1]:
def checkMenu(supergoods=[]):
    '''Prints all available recipes'''
    info = getInfo()
    for i,x in enumerate(info):
        if x[0] in supergoods:
            print(str(i+1) + '. ' + '                                                                       ' + str(x[0]))
        else:
            print(str(i+1) + '. ' + str(x[0]))

def getInfo():
    '''Pulls from grocery info, which is formatted as recipe|ingredient 1,ingredient 2,...
    Output is a list of [recipe name,[[quantity,ingredient name 1],[quantity,ingredient name 2]]]'''
    with open('./_INFO.txt','r') as f:
        info = f.readlines()
        info = [line.split('|') for line in info] # Sometimes I'll leave inspection hints in the _INFO.txt file as well
        
        def sliceIngredients(ingredients):
            ingredients = ingredients.split(',')
            ingredients = [[x[:x.index(' ')],x[x.index(' ')+1:].strip('\n')] for x in ingredients]
            # Had to strip \n because it was keeping it from the end of lines
            return ingredients
        info = [[line[0],sliceIngredients(line[1])] for line in info]
        
    return info

def toIngredients(user_request):
    '''Takes in a list of [recipe name, quantity]
    Looks up ingredients in _INFO.txt, gets ingredients and adds them together.
    If it can\'t find a recipe, it will output that as an error, but will continue adding the rest of the recipes together.'''
    print('Total meals requested: ' + str(sum([x[1] for x in user_request])))
    info = getInfo() # Get "meal name":[ingredient list] 
    master_meals = [x[0] for x in info]
    ingredients, amounts, errors = [],[],[] # First one is list of strings, second is list of lists (of strings)
    
    # Expand (for example) 5 dan dan noodles to a 5 long list of ['dan dan noodles','dan dan noodles',...]
    # This is easier than trying to multiply the number parts of the ingredients
    new = []
    for meal in user_request:
        new += [[meal[0],1]]*meal[1]
    user_request = new
    
    for meal in user_request: # Check if meals are in the master meal list
        if meal[0] in master_meals: # If they are...
            loc = master_meals.index(meal[0])
            for part in info[loc][1]: # Put the ingredients on the amounts array. (part being ['18oz','ground pork'])
                if part[1] in ingredients:
                    amounts[ingredients.index(part[1])].append(part[0])
                else:
                    ingredients.append(part[1])
                    amounts.append([part[0]])
        else: # If they aren't, add them to the errors list.
            errors.append(meal[0])
    
    print('Finished looking up ingredients!\nLookup errors: ' + str(len(errors)))
    if len(errors) > 0:
        print(errors)
    
    return ingredients,amounts,errors

def unit_sum(amounts):
    '''Input: amounts list from toIngredients()
    Output: ingredient quantity sum list, in best quantity mentioned. Rounds to 2 decimal places in end.'''
    def quantity_unit(amounts): # Technically not good practice, but I don't modify the amounts list
        '''Splits list of ['1tbsp',...] into [[1,'tbsp'],...]
        Technically, it\'s doing the entire amounts array from toIngredients(), which adds a bit of additional complexity.'''
        numbers = {'0','1','2','3','4','5','6','7','8','9','.'} # Decimal place so it properly deals with '0.25'
        # These next lines could potentially be a bit confusing, but it's just creating a blank list for each ingredient value in the amounts list.
        # These lists will be later be replaced by lists like [1,'tbsp']
        sub_lengths = [len(x) for x in amounts]
        split_array = [[[]]*sub_lengths[x] for x in range(len(amounts))]
        
        for amount_index, ingredient in enumerate(amounts):
            for ingredient_index, sub_ingredient in enumerate(ingredient):
                number_part = ''.join([x for x in list(sub_ingredient) if x in numbers])
                if number_part == '': # Sometimes "pinch" is listed as an amount. I'm listing it as 0 so the indexes line up.
                    number_part = 0
                else:
                    number_part = float(number_part)
                
                string_part = ''.join([x for x in list(sub_ingredient) if x not in numbers])
                
                if string_part == 'cup':
                    string_part = 'cups' # Adding consistency
                
                split_array[amount_index][ingredient_index] = [number_part,string_part]
        return split_array
    
    preferred_units = ['tbsp','lb','oz','cups','tsp'] # In that order
    relationships = {
        # I could make something that keeps track of both sides of the relationship,
        # but there aren't that many kitchen quantities. 
        'cupsoz': 8,
        'tsptbsp': 0.33,
        'lboz': 16,
        'cupstbsp': 16,
        'oztbsp': 2,
        'ozlb': 0.125,
        'tspcups': 0.021
        # 'sliceoz': 1 # Specifically for thick cut bacon
    }
    
    unit_error_indexes = [] # Since this function doesn't have access to the names, just the indexes of errors, it'll save them for print output later.
    new_amounts = quantity_unit(amounts)
    sums = [[]]*len(new_amounts)
    for amount_index,ingredient in enumerate(new_amounts):
        # Scan over the sub_ingredients and choose a suitable unit
        # This looks for the lowest index in preferred_units and sets the default to that.
        # It's not a perfect system but quite good nonetheless.
        indexes = [0]*len(ingredient)
        for ingredient_index, sub_ingredient in enumerate(ingredient):
            try:
                index = preferred_units.index(sub_ingredient[1])
            except ValueError:
                index = 10 # Outside the bounds of preferred_units. Technically this would be better set to np.inf or something for extensibility
            indexes[ingredient_index] = index
        if min(indexes) < len(preferred_units):
            unit = preferred_units[min(indexes)]
        else:
            unit = ingredient[0][1]
        # Note: The above is tolerant to unnamed quantities, like 3 cabbages. This is because the unit type is set to ''
        totalsum = 0
        for sub_ingredient in ingredient:
            if sub_ingredient[1] == unit: # If it's the chosen unit, just add it to the running sum
                totalsum += sub_ingredient[0]
            else: # If it's not the chosen unit, look up the conversion and then add it
                if sub_ingredient[1] == 'pinch': # Pinch adds nothing to the final amount
                    pass
                else:
                    try:
                        totalsum += relationships[sub_ingredient[1] + unit] # Includes info on unit from and unit to
                    except KeyError: # Can't find unit conversion
                        # print('Can\'t find unit conversion!', '"' + sub_ingredient[1] + '" to "' + unit + '"','\n',ingredient)
                        unit_error_indexes.append(amount_index)
        sums[amount_index] = [totalsum,unit]
    sums = [[round(x[0],2),x[1]] for x in sums]
    
    return sums, unit_error_indexes

In [4]:
# Yes to double portion sizes :D
finals = toIngredients([['Easy Sesame Chicken',2], # Up to 4 at a time
                        # ['Asian Noodle Bowl with Poached Egg',1] # If noodles made properly, can do up to 2 at a time
                        # ['Dan Dan Noodles',2], # Have to use pressure cooker to do two at a time
                        # ['Easy Egg Fried Rice',2],
                        ['Lighter Egg Foo Young', 2]
                        # ['Herbed Frittata with Cherry Tomatoes',4], # Can make 4 at a time with two trays
                        # ['Shakshuka with Feta and Farro',2] # Can make two at a time because cast iron
                       ])
sums, unit_error_indexes = unit_sum(finals[1])
print('Unit conversion errors:',len(unit_error_indexes))
print('--------------------------------------------------------------')
for i in range(len(finals[0])):
    print('{:<20} {}'.format(finals[0][i],str(sums[i][0]) + ' ' + str(sums[i][1])),end=' ')
    if i in unit_error_indexes:
        print(finals[1][i])
    else:
        print('')
# Notes:
    # About 1 cup (~200g) of farro per 2 cups of cooked farro

Total meals requested: 4
Finished looking up ingredients!
Lookup errors: 0
Unit conversion errors: 0
--------------------------------------------------------------
chicken              2.0 lb 
soy sauce            4.66 tbsp 
sesame oil           4.0 tsp 
sugar                2.0 tbsp 
honey                2.0 tbsp 
rice vinegar         4.0 tbsp 
ginger               2.0 tbsp 
garlic clove         2.0  
eggs                 14.0  
cornstarch           8.0 tbsp 
chicken broth        1.5 cups 
hoisin sauce         3.0 tbsp 
peanut oil           6.0 tbsp 
cremini mushrooms    8.0  
scallions            8.0  
bean sprouts         3.0 cups 
bacon                0.5 cups 


In [3]:
checkMenu(supergoods=['Dan Dan Noodles',
           'Shrimp with Lobster Sauce',
           'Egg Drop Soup',
           'Lighter Egg Foo Young',
           'Easy Sesame Chicken',
           'Easy Egg Fried Rice',
           'Easy Chicken Congee with Peanuts',
           'Herbed Frittata with Cherry Tomatoes',
           'Shakshuka with Feta and Farro',
           'Asian Noodle Bowl with Poached Egg',
           'Eggs Blackstone'])

1.                                                                        Dan Dan Noodles
2. Sweet Chili Shrimp
3. Shrimp With Lobster Sauce
4.                                                                        Egg Drop Soup
5.                                                                        Lighter Egg Foo Young
6. Vinegar-Glazed Chinese Cabbage
7. Hot and Sour Vegetable Soup
8. Vegetable Egg Rolls
9. Pineapple Fried Rice
10. Easy Vegetable Stir Fry
11. Mango Pudding
12. Teriyaki Turkey Rice Bowl
13. Chicken and Ginger Noodle Soup
14. Braised Pork in Soy Sauce
15. General Tso's Chicken
16.                                                                        Easy Sesame Chicken
17. Breakfast Burrito
18. Orange French Toast
19. Quick Fried Wontons
20. Velvet Chicken
21.                                                                        Easy Egg Fried Rice
22.                                                                        Easy Chicken Congee with Peanuts
23. Twice

In [3]:
# Used to check an individual item on the menu
request = 'Shakshuka with Feta and Farro'

### Boilerplate below here ###
finals = toIngredients([[request,1]])
# print('--------------------------------------------------------------')
sums, unit_error_indexes = unit_sum(finals[1])
print('Unit conversion errors:',len(unit_error_indexes))
print('--------------------------------------------------------------')
for i in range(len(finals[0])):
    print('{:<20} {}'.format(finals[0][i],str(sums[i][0]) + ' ' + str(sums[i][1])),end=' ')
    if i in unit_error_indexes:
        print(finals[1][i])
    else:
        print('')

Total meals requested: 1
Finished looking up ingredients!
Lookup errors: 0
Unit conversion errors: 0
--------------------------------------------------------------
olive oil            2.0 tbsp 
yellow onion         0.5  
garlic               1.0 clove 
red bell pepper      1.0  
ground cumin         2.0 tsp 
smoked paprika       0.75 tsp 
red pepper flakes    0.25 tsp 
diced tomatoes       28.0 oz 
feta cheese          0.25 lb 
eggs                 6.0  
farro                3.0 cups 
fresh parsley        2.0 tbsp 


In [27]:
### Used for Arkansas visit
finals = toIngredients([['Eggs Blackstone',2]])
# print('--------------------------------------------------------------')
sums, unit_error_indexes = unit_sum(finals[1])
print('Unit conversion errors:',len(unit_error_indexes))
print('--------------------------------------------------------------')
for i in range(len(finals[0])):
    print('{:<20} {}'.format(finals[0][i],str(sums[i][0]) + ' ' + str(sums[i][1])),end=' ')
    if i in unit_error_indexes:
        print(finals[1][i])
    else:
        print('')

Total meals requested: 2
Finished looking up ingredients!
Lookup errors: 0
Unit conversion errors: 0
--------------------------------------------------------------
tomato               8.0  
olive oil            2.0 tbsp 
fresh thyme          2.0 tsp 
bacon                16.0 slice 
country bread        16.0  
eggs                 24.0  
lemon juice          4.0 tbsp 
butter               2.0 cups 


In [1]:
# 20190116 Available
# Dan Dan Noodles
# Egg Drop Soup
# Asian Noodle Bowl w/ Poached Egg
# Egg Foo Young
# Eggs Blackstone

In [4]:
[x[0] for x in getInfo()]

['Dan Dan Noodles',
 'Sweet Chili Shrimp',
 'Shrimp With Lobster Sauce',
 'Egg Drop Soup',
 'Lighter Egg Foo Young',
 'Vinegar-Glazed Chinese Cabbage',
 'Hot and Sour Vegetable Soup',
 'Vegetable Egg Rolls',
 'Pineapple Fried Rice',
 'Easy Vegetable Stir Fry',
 'Mango Pudding',
 'Teriyaki Turkey Rice Bowl',
 'Chicken and Ginger Noodle Soup',
 'Braised Pork in Soy Sauce',
 "General Tso's Chicken",
 'Easy Sesame Chicken',
 'Breakfast Burrito',
 'Orange French Toast',
 'Quick Fried Wontons',
 'Velvet Chicken',
 'Easy Egg Fried Rice',
 'Easy Chicken Congee with Peanuts',
 'Twice-Cooked Pork',
 'Succulent Shrimp Wontons',
 'Stir-Fried Eggs with Tomato',
 'Herbed Frittata with Cherry Tomatoes',
 'Shakshuka with Feta and Farro',
 'Asian Noodle Bowl with Poached Egg',
 'Poached Eggs',
 'Eggs Blackstone']

In [6]:
finals = toIngredients([[x[0],1] for x in getInfo()]) 
# print('--------------------------------------------------------------')
sums, unit_error_indexes = unit_sum(finals[1])
print('Unit conversion errors:',len(unit_error_indexes))
print('--------------------------------------------------------------')
for i in range(len(finals[0])):
    print('{:<20} {}'.format(finals[0][i],str(sums[i][0]) + ' ' + str(sums[i][1])),end=' ')
    if i in unit_error_indexes:
        print(finals[1][i])
    else:
        print('')

Total meals requested: 30
Finished looking up ingredients!
Lookup errors: 0
Unit conversion errors: 14
--------------------------------------------------------------
peanut oil           28.33 tbsp 
bacon                1.12 lb ['16oz', '0.25cups', '0.25cups', '0.5cups', '1lb', '4slice', '8slice']
ginger               25.65 tbsp 
chicken broth        25.62 cups 
chili sauce          5.33 tbsp 
rice vinegar         7.5 tbsp 
soy sauce            46.32 tbsp 
peanut butter        3.0 tbsp 
sichuan peppercorns  1.5 tsp 
salt                 1.66 tsp 
pepper               0.25 tsp 
egg noodles          12.0 oz 
roasted peanuts      3.0 tbsp 
scallions            33.5  
cellophane noodles   8.0 oz 
shrimp               1.46 lb 
cornstarch           9.32 tbsp 
black pepper         0.38 tsp 
honey                4.5 tbsp 
Shaoxing rice wine   34.99 tbsp 
minced garlic        2.0 tbsp 
garlic cloves        15.0  
sesame oil           3.31 tbsp 
sugar                27.31 tbsp 
white pepper     